In [6]:
import pandas as pd

def evaluate_robustness(df, policy_id_col, scenario_id_col, performance_cols, satisficing_thresholds):
    grouped = df.groupby(policy_id_col)
    results = []

    # Prepare rank matrix
    rank_dfs = {}
    for col in performance_cols:
        rank_df = df.copy()
        rank_df['Rank'] = rank_df.groupby(scenario_id_col)[col].rank(method='min')
        rank_dfs[col] = rank_df[[policy_id_col, scenario_id_col, 'Rank']]

    for policy, group in grouped:
        metrics = {'Policy': policy}

        for col in performance_cols:
            values = group[col]

            metrics[f'{col} Mean'] = values.mean()
            metrics[f'{col} Std'] = values.std()
            metrics[f'{col} CV'] = values.std() / values.mean() if values.mean() != 0 else float('inf')

            # Regret analysis
            scenario_best = df.groupby(scenario_id_col)[col].min()
            joined = group[[scenario_id_col, col]].merge(scenario_best.rename('Best'), on=scenario_id_col)
            joined['Regret'] = joined[col] - joined['Best']

            metrics[f'{col} Max Regret'] = joined['Regret'].max()
            metrics[f'{col} Mean Regret'] = joined['Regret'].mean()

            # Rank consistency
            policy_ranks = rank_dfs[col][rank_dfs[col][policy_id_col] == policy]['Rank']
            metrics[f'{col} Rank Std'] = policy_ranks.std()

        # Satisficing check
        satisficing_count = 0
        for _, row in group.iterrows():
            if all(row[c] <= satisficing_thresholds[c] for c in satisficing_thresholds):
                satisficing_count += 1

        metrics['Satisficing Count'] = satisficing_count
        metrics['Total Scenarios'] = len(group)

        results.append(metrics)

    robustness_df = pd.DataFrame(results)
    return robustness_df

# === LOAD & PREPARE DATA ===

# Load the CSV
df = pd.read_csv(r"C:\Users\jesse\MSc_EPA_VSCode\EPA_MBDM_Project\EPA141A\final assignment\results\robustness_test_dataframe_8-6-25.csv")

# Add 'Total Costs' column
df['Total Costs'] = (
    df['Expected Annual Damage'] +
    df['Dike Investment Costs'] +
    df['RfR Investment Costs'] +
    df['Evacuation Costs']
)

# Define the performance columns
performance_columns = [
    'Expected Annual Damage',
    'Dike Investment Costs',
    'RfR Investment Costs',
    'Evacuation Costs',
    'Expected Number of Deaths',
    'Total Costs'
]

# Define satisficing thresholds (adjust to your preferences)
thresholds = {
    'Expected Annual Damage': 1e7,
    'Dike Investment Costs': 1e8,
    'RfR Investment Costs': 1e7,
    'Evacuation Costs': 1e6,
    'Expected Number of Deaths': 5,
    'Total Costs': 1.5e8
}

# Run the robustness analysis
robustness_summary = evaluate_robustness(
    df,
    policy_id_col='policy',
    scenario_id_col='scenario',
    performance_cols=performance_columns,
    satisficing_thresholds=thresholds
)

# Save the results
robustness_summary.to_csv("policy_robustness_with_all_indicators.csv", index=False)
print("✅ Robustness summary saved as 'policy_robustness_with_all_indicators.csv'")


✅ Robustness summary saved as 'policy_robustness_with_all_indicators.csv'
